# Homework 4

In [11]:
import numpy as np
import urllib
import string
import nltk
from nltk import bigrams
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from scipy import spatial
from collections import defaultdict

In [2]:
def parseData(fname):
    for l in urllib.urlopen(fname):
        yield eval(l)

print "Reading data..."
data = list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))[:5000]
print "done"

Reading data...
done


In [3]:
data[0]

{'beer/ABV': 5.0,
 'beer/beerId': '47986',
 'beer/brewerId': '10325',
 'beer/name': 'Sausa Weizen',
 'beer/style': 'Hefeweizen',
 'review/appearance': 2.5,
 'review/aroma': 2.0,
 'review/overall': 1.5,
 'review/palate': 1.5,
 'review/taste': 1.5,
 'review/text': 'A lot of foam. But a lot.\tIn the smell some banana, and then lactic and tart. Not a good start.\tQuite dark orange in color, with a lively carbonation (now visible, under the foam).\tAgain tending to lactic sourness.\tSame for the taste. With some yeast and banana.',
 'review/timeStruct': {'hour': 20,
  'isdst': 0,
  'mday': 16,
  'min': 57,
  'mon': 2,
  'sec': 3,
  'wday': 0,
  'yday': 47,
  'year': 2009},
 'review/timeUnix': 1234817823,
 'user/profileName': 'stcules'}

# Task 1
The number of unique bigram is 182246

The 5 most-frequently-occurring bigrams along with their number of occurrences in the corpus:

('with', 'a') : 4587

('in', 'the') : 2595

('of', 'the') : 2245

('is', 'a') : 2056

('on', 'the') : 2033

In [12]:
from string import punctuation
print punctuation

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [30]:
bigram = defaultdict(int)
allText = [x['review/text'] for x in data]
for text in allText:
    textList = text.translate(None, punctuation).lower().split()
    for i in range(0,len(textList)-1):
        bigram[tuple(textList[i:i+2])] += 1
import operator
sortedBigram = sorted(bigram.items(), key=operator.itemgetter(1), reverse=True)

In [109]:
print len(bigram.keys())

182246


In [34]:
sortedBigram[:5]

[(('with', 'a'), 4587),
 (('in', 'the'), 2595),
 (('of', 'the'), 2245),
 (('is', 'a'), 2056),
 (('on', 'the'), 2033)]

# Task 2
0.34361068509441478

In [35]:
popularBigram = [x[0] for x in sortedBigram[:1000]]
popularBigramToID= dict(zip(popularBigram, range(len(popularBigram))))
popularBigramSet = set(popularBigram)

def feature(datum):
    feat = [0]*len(popularBigram)
    r = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
    r = r.split()
    for i in range(0, len(r)):
        if tuple(r[i:i+2]) in popularBigram:
            feat[popularBigramToID[tuple(r[i:i+2])]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in data]
y = [d['review/overall'] for d in data]

#With regularization
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)
mean_squared_error(predictions, y)

0.34361068509441478

# Task 3
The MSE obtained using unigrams and bigrams is : 0.28933386918744819

In [60]:
unigram = defaultdict(int)
for text in allText:
    textList = text.translate(None, punctuation).lower().split()
    for w in textList:
        unigram[w] += 1

In [61]:
uniAndBi = unigram.copy()
uniAndBi.update(bigram)
sortedUniAndBi = sorted(uniAndBi.items(), key=operator.itemgetter(1), reverse=True)

In [62]:
allGrams = [x[0] for x in sortedUniAndBi[:1000]]
allGramID = dict(zip(allGrams, range(len(allGrams))))
allGramSet = set(allGrams)

def feature(datum):
    feat = [0] * len(allGrams)
    text = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
    text = text.split()
    for i in range(0,len(text)):
        if text[i] in allGramSet:
            feat[allGramID[text[i]]] += 1
        if i < len(text)-1 and tuple(text[i:i+2]) in allGramSet:
            feat[allGramID[tuple(text[i:i+2])]] += 1
    feat.append(1) #offset
    return feat

In [63]:
x = [feature(d) for d in data]
y = [d['review/overall'] for d in data]

clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(x, y)
theta = clf.coef_
predictions = clf.predict(x)
mean_squared_error(predictions, y)

0.28933386918744819

# Task 4
The 5 unigrams/bigrams with the most positive associated weights are:

'sort', 0.52168077682148783

('a', 'bad'), 0.22628883434803632

('of', 'these'), 0.22289001188018825

('not', 'bad'), 0.21626861571067457 

('the', 'best'), 0.21377221903566851

 The 5 unigrams/bigrams with the most negative associated weights are : 

('sort', 'of'), -0.64593794533391358

'water', -0.27190017695088031

'corn', -0.23756003904023357

('the', 'background'), -0.21813867244801693

'straw', -0.19975354891661662

In [71]:
weights = zip(theta[:-1], range(1000))
weights = map(lambda x: (allGrams[x[1]],x[0]), weights)
weights.sort(key = lambda x:x[1], reverse = True)

In [73]:
print 'Positive: ', weights[:5]
print 'Negative: ', weights[-5:][::-1]

Positive:  [('sort', 0.52168077682148783), (('a', 'bad'), 0.22628883434803632), (('of', 'these'), 0.22289001188018825), (('not', 'bad'), 0.21626861571067457), (('the', 'best'), 0.21377221903566851)]
Negative:  [(('sort', 'of'), -0.64593794533391358), ('water', -0.27190017695088031), ('corn', -0.23756003904023357), (('the', 'background'), -0.21813867244801693), ('straw', -0.19975354891661662)]


# Task 5
The inverse document frequency and tf-idf of words 'foam', 'smell', 'banana', 'lactic', and 'tart' are :

foam idf: 1.13786862069 tfidf: 2.27573724137

smell idf: 0.537901618865 tfidf: 0.537901618865

banana idf: 1.67778070527 tfidf: 3.35556141053

lactic idf: 2.92081875395 tfidf: 5.8416375079

tart idf: 1.80687540165 tfidf: 1.80687540165

In [76]:
def tf(word, text):
    text = text.translate(None, punctuation).lower().split()
    return text.count(word)
def idf(word, allText):
    hasWord = filter(lambda text: word in text.translate(None, punctuation).lower().split(), allText)
    return np.log10(len(allText)*1./len(hasWord))

In [78]:
for word in ['foam', 'smell', 'banana', 'lactic', 'tart']:
    wordidf = idf(word, allText)
    print word, "idf:", wordidf, "tfidf:", tf(word,data[0]['review/text']) * wordidf

foam idf: 1.13786862069 tfidf: 2.27573724137
smell idf: 0.537901618865 tfidf: 0.537901618865
banana idf: 1.67778070527 tfidf: 3.35556141053
lactic idf: 2.92081875395 tfidf: 5.8416375079
tart idf: 1.80687540165 tfidf: 1.80687540165


# Task 6
The cosine similarity between the first and the second review in terms of their tf-idf representations is : 0.106130241679

In [105]:
sortedUnigram = sorted(unigram.items(), key=operator.itemgetter(1), reverse=True)
popularUnigram = [x[0] for x in sortedUnigram[:1000]]
popularUnigramID = dict(zip(popularUnigram, range(len(popularUnigram))))
allTextList = [text.translate(None, punctuation).lower().split() for text in allText]
wordIdf = defaultdict(float)
for l in allTextList:
    for word in popularUnigram:
        if word in l:
            wordIdf[word] += 1
for word in popularUnigram:
    wordIdf[word] = np.log10(len(allText)*1./wordIdf[word])
                             
def tfidfFeature(text):
    feature = [0]* 1000
    textList = text.translate(None, punctuation).lower().split()
    wordCount = defaultdict(int)
    for word in textList:
        wordCount[word] += 1
    for i, word in enumerate(popularUnigram):
        feature[i] = wordCount[word]*wordIdf[word]
    return feature

In [106]:
feature1 = tfidfFeature(data[0]['review/text'])
feature2 = tfidfFeature(data[1]['review/text'])
print 1 - spatial.distance.cosine(feature1, feature2)

0.106130241679


# Task 7
beerID : 52211

profileName : Dope

reviewText : A: A hazy deep orange pour, almost red. Small white head that fades quickly. A little spotty lacing.		S: Big pumpkin, cinnamon, ginger, nutmeg and brown sugar. Sweet. Smells like a pumpkin pie mixed with a gingerbread cookie.		T: Tons of pumpkin dominates throughout. Cinnamon, ginger, nutmeg and a bit of vanilla creaminess. 		M: Smooth medium body. Tiny bit of drying alcohol.		O: Excellent pumpkin ale. Heavy on the pumpkin but the spices don't get completely overshadowed either.

In [102]:
cosineDistanceList = [1 - spatial.distance.cosine(feat1, tfidfFeature(x['review/text'])) for x in data[1:]]
result = zip(cosineDistanceList, range(len(cosineDistanceList)))
result.sort(key = lambda x:x[0], reverse = True)
print 'beerID : ', data[result[0][1]]['beer/beerId']
print 'profileName : ', data[result[0][1]]['user/profileName']
print 'reviewText : ', data[result[0][1]]['review/text']

beerID :  52211
profileName :  Dope
reviewText :  A: A hazy deep orange pour, almost red. Small white head that fades quickly. A little spotty lacing.		S: Big pumpkin, cinnamon, ginger, nutmeg and brown sugar. Sweet. Smells like a pumpkin pie mixed with a gingerbread cookie.		T: Tons of pumpkin dominates throughout. Cinnamon, ginger, nutmeg and a bit of vanilla creaminess. 		M: Smooth medium body. Tiny bit of drying alcohol.		O: Excellent pumpkin ale. Heavy on the pumpkin but the spices don't get completely overshadowed either.


# Task 8
The MSE obtained with the 1000-dimensional tf-idf representations is : 0.278759560078

In [116]:
def feature(datum):
    feat = [0]*len(popularUnigram)
    r = ''.join([c for c in datum['review/text'].lower() if not c in punctuation])
    for w in r.split():
        if w in set(popularUnigram):
            feat[popularUnigramID[w]] += 1
    for i, word in enumerate(popularUnigram):
        feat[i] *= wordIdf[word]
    feat.append(1) #offset
    return feat

X = [feature(d) for d in data]
y = [d['review/overall'] for d in data]

#No regularization
#theta,residuals,rank,s = numpy.linalg.lstsq(X, y)

#With regularization
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)
mean_squared_error(predictions, y)

0.278759560078
